In [1]:
# install it if you do not have
# pip install kagglehub
import kagglehub
import os, shutil


import numpy as np
import pandas as pd

# Download latest version
csv_name = "WA_Fn-UseC_-Telco-Customer-Churn.csv"
path_download = kagglehub.dataset_download("blastchar/telco-customer-churn",force_download=True)
file_download = os.path.join(path_download, csv_name)

#-------------------------------------------------------------------------------------------------------
# Instead of saving to Google Drive directly, let's save to a local Colab directory first
#-------------------------------------------------------------------------------------------------------
path_destination = "dataset"
file_destination = os.path.join(path_destination, csv_name)

os.makedirs(path_destination,exist_ok=True)

# Check if the destination file already exists
if os.path.exists(file_destination):
    os.remove(file_destination)

# Use shutil.copyfile instead of shutil.move because the source is on a read-only file system
shutil.copyfile(file_download,file_destination)

df_rawtelco = pd.read_csv(file_destination)

pd.options.display.max_columns = None
df_rawtelco.head()

Using Colab cache for faster access to the 'telco-customer-churn' dataset.


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
## Exploring and understanding the data ##

print("Shape:",df_rawtelco.shape)
df_rawtelco.info()

Shape: (7043, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 n

In [3]:
## target distribution

df_rawtelco["Churn"].value_counts(normalize=True) * 100

,proportion
Churn,
No,73.463013
Yes,26.536987


In [4]:
df_rawtelco.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [5]:
## missing values

df_rawtelco.isna().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [6]:
## Data cleaning

## keep the original data
df_cleantelco = df_rawtelco.copy(deep = True)

## Total charges - Replace spaces with np.nan and converto to numeric datatype.

df_cleantelco["TotalCharges"] = df_cleantelco["TotalCharges"].replace(" ",np.nan )
df_cleantelco["TotalCharges"] = pd.to_numeric(df_cleantelco["TotalCharges"])

df_cleantelco["TotalCharges"].isna().sum()

np.int64(11)

In [7]:
## delete rows with totalchages null - small number of rows (11 only)

df_cleantelco = df_cleantelco.dropna(subset = ["TotalCharges"])
df_cleantelco.shape


(7032, 21)

In [8]:
# drop column customerid
df_cleantelco = df_cleantelco.drop(columns=["customerID"])

# convert Churn to binary
df_cleantelco["Churn"] = df_cleantelco["Churn"].map({"Yes":1, "No":0})

df_cleantelco["Churn"].value_counts()


,count
Churn,
0,5163
1,1869


In [9]:
cols_to_simplify = [
    "MultipleLines", "OnlineSecurity", "OnlineBackup", "DeviceProtection",
    "TechSupport", "StreamingTV", "StreamingMovies"
]

for col in cols_to_simplify:
    df_cleantelco[col] = df_cleantelco[col].replace({"No internet service": "No"})

# For phone service columns
df_cleantelco["MultipleLines"] = df_cleantelco["MultipleLines"].replace({"No phone service": "No"})

In [10]:
df_cleantelco.head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1
5,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,1
6,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,0
7,Female,0,No,No,10,No,No,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.90,0
8,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1
9,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,0


### **Encode Categorical Features:**
Generally, it is required to apply One-Hot Encoding (or another suitable encoding) to the categorical features before applying machine learning models like Logistic Regression, Random Forest Classifier, and Gradient Boost. Here's why:

*   **Logistic Regression:** This is a linear model and requires numerical input. Categorical variables must be converted into a numerical format for the model to process them correctly.

*   **Random Forest Classifier:** While tree-based models can sometimes implicitly handle categorical data, many implementations (including scikit-learn's Random Forest) still expect numerical input. One-Hot Encoding prevents the model from assuming an ordinal relationship between categories.
*   **Gradient Boost (e.g., XGBoost, LightGBM):** Similar to Random Forest, these models often perform best or require numerical input. Some advanced Gradient Boosting libraries like CatBoost can handle categorical features natively, but for more common implementations like XGBoost or LightGBM, encoding is usually necessary.

One-Hot Encoding creates new binary (0 or 1) columns for each category, which is a robust way to represent nominal categorical data without implying any false ordinality.

In [11]:
# Binning

dep_var = ['Churn']

categorical_vars = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
                    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
                    'PaperlessBilling', 'PaymentMethod']

Numeric_vars = ['tenure', 'MonthlyCharges', 'TotalCharges']


In [12]:
# Categorical variable - we will examin the number of occurances of each category.

for item in categorical_vars:
    print(df_cleantelco[item].value_counts())
    print()

gender
Male      3549
Female    3483
Name: count, dtype: int64

SeniorCitizen
0    5890
1    1142
Name: count, dtype: int64

Partner
No     3639
Yes    3393
Name: count, dtype: int64

Dependents
No     4933
Yes    2099
Name: count, dtype: int64

PhoneService
Yes    6352
No      680
Name: count, dtype: int64

MultipleLines
No     4065
Yes    2967
Name: count, dtype: int64

InternetService
Fiber optic    3096
DSL            2416
No             1520
Name: count, dtype: int64

OnlineSecurity
No     5017
Yes    2015
Name: count, dtype: int64

OnlineBackup
No     4607
Yes    2425
Name: count, dtype: int64

DeviceProtection
No     4614
Yes    2418
Name: count, dtype: int64

TechSupport
No     4992
Yes    2040
Name: count, dtype: int64

StreamingTV
No     4329
Yes    2703
Name: count, dtype: int64

StreamingMovies
No     4301
Yes    2731
Name: count, dtype: int64

Contract
Month-to-month    3875
Two year          1685
One year          1472
Name: count, dtype: int64

PaperlessBilling
Yes    41

In [13]:
# Categorical Encoding is necessary for a model to work with categorical data

# categorical_vars = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
#                     'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
#                     'PaperlessBilling', 'PaymentMethod']

from sklearn.preprocessing import OneHotEncoder


tfmr = OneHotEncoder(sparse_output=False)

print(df_cleantelco[['gender']].values)
print(tfmr.fit_transform(df_cleantelco[['gender']].values))

print(df_cleantelco[['SeniorCitizen']].values)
print(tfmr.fit_transform(df_cleantelco[['SeniorCitizen']].values))

print(df_cleantelco[['Partner']].values)
print(tfmr.fit_transform(df_cleantelco[['Partner']].values))

print(df_cleantelco[['Dependents']].values)
print(tfmr.fit_transform(df_cleantelco[['Dependents']].values))

print(df_cleantelco[['PhoneService']].values)
print(tfmr.fit_transform(df_cleantelco[['PhoneService']].values))

print(df_cleantelco[['MultipleLines']].values)
print(tfmr.fit_transform(df_cleantelco[['MultipleLines']].values))

print(df_cleantelco[['InternetService']].values)
print(tfmr.fit_transform(df_cleantelco[['InternetService']].values))

print(df_cleantelco[['OnlineSecurity']].values)
print(tfmr.fit_transform(df_cleantelco[['OnlineSecurity']].values))

print(df_cleantelco[['OnlineBackup']].values)
print(tfmr.fit_transform(df_cleantelco[['OnlineBackup']].values))

print(df_cleantelco[['DeviceProtection']].values)
print(tfmr.fit_transform(df_cleantelco[['DeviceProtection']].values))

print(df_cleantelco[['TechSupport']].values)
print(tfmr.fit_transform(df_cleantelco[['TechSupport']].values))

print(df_cleantelco[['StreamingTV']].values)
print(tfmr.fit_transform(df_cleantelco[['StreamingTV']].values))

print(df_cleantelco[['StreamingMovies']].values)
print(tfmr.fit_transform(df_cleantelco[['StreamingMovies']].values))

print(df_cleantelco[['Contract']].values)
print(tfmr.fit_transform(df_cleantelco[['Contract']].values))

print(df_cleantelco[['PaperlessBilling']].values)
print(tfmr.fit_transform(df_cleantelco[['PaperlessBilling']].values))

print(df_cleantelco[['PaymentMethod']].values)
print(tfmr.fit_transform(df_cleantelco[['PaymentMethod']].values))


[['Female']
 ['Male']
 ['Male']
 ...
 ['Female']
 ['Male']
 ['Male']]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]
[[0]
 [0]
 [0]
 ...
 [0]
 [1]
 [0]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
[['Yes']
 ['No']
 ['No']
 ...
 ['Yes']
 ['Yes']
 ['No']]
[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]
[['No']
 ['No']
 ['No']
 ...
 ['Yes']
 ['No']
 ['No']]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[['No']
 ['Yes']
 ['Yes']
 ...
 ['No']
 ['Yes']
 ['Yes']]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]
[['No']
 ['No']
 ['No']
 ...
 ['No']
 ['Yes']
 ['No']]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
[['DSL']
 ['DSL']
 ['DSL']
 ...
 ['DSL']
 ['Fiber optic']
 ['Fiber optic']]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
[['No']
 ['Yes']
 ['Yes']
 ...
 ['Yes']
 ['No']
 ['Yes']]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
[['Yes']
 ['No']
 ['Yes']
 ...
 ['No'

In [14]:
################################################
# Categorical Transformer - one hot encoder
################################################
# For our categorical features, we can use one-hot encoding to represent the different categories numerically.
# With this approach, each unique value of a feature is given its own column. Thus, the data will be encoded with a "1"
# in the column which matches the value and a "0" in every other column.
# The use of multiple columns via one-hot encoding ensures that our categorical values are not treated as ordinal data.

# "Categorical Encoder" Transformer - (name, transformer) tuple
# Removed the intermediate Pipeline and directly use OneHotEncoder in the transformer list for ColumnTransformer.
cat_transformers = [('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_vars)]

In [15]:
################################################
# No Transformation of Numeric independent variables
################################################
from sklearn.compose import ColumnTransformer
#from sklearn.pipeline import Pipeline

# ColumnTransformer
ct = ColumnTransformer(transformers=cat_transformers + [('passthrough', 'passthrough',Numeric_vars)])
ct.fit(df_cleantelco[categorical_vars + Numeric_vars])
X = ct.transform(df_cleantelco[categorical_vars + Numeric_vars])
# We know from our exploration that the dependent variable 'Churn' does not have any missing values.
y = df_cleantelco[['Churn']].values

In [17]:
print(X)
print()
print(y)

[[1.0000e+00 0.0000e+00 1.0000e+00 ... 1.0000e+00 2.9850e+01 2.9850e+01]
 [0.0000e+00 1.0000e+00 1.0000e+00 ... 3.4000e+01 5.6950e+01 1.8895e+03]
 [0.0000e+00 1.0000e+00 1.0000e+00 ... 2.0000e+00 5.3850e+01 1.0815e+02]
 ...
 [1.0000e+00 0.0000e+00 1.0000e+00 ... 1.1000e+01 2.9600e+01 3.4645e+02]
 [0.0000e+00 1.0000e+00 0.0000e+00 ... 4.0000e+00 7.4400e+01 3.0660e+02]
 [0.0000e+00 1.0000e+00 1.0000e+00 ... 6.6000e+01 1.0565e+02 6.8445e+03]]

[[0]
 [0]
 [1]
 ...
 [0]
 [1]
 [0]]


In [19]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (7032, 39)
Shape of y: (7032, 1)


### **Train / Test Split:**

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data preparation and splitting complete.")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

Data preparation and splitting complete.
X_train shape: (5625, 39)
X_test shape: (1407, 39)
y_train shape: (5625, 1)
y_test shape: (1407, 1)


### **Dimension Reduction:**

PCA doesn't directly 'retain' original columns. Instead, it transforms data into a new set of dimensions called principal components, which are linear combinations of original features.

To understand which original columns contribute most to these principal components, examine the components_ attribute of the fitted PCA object. This attribute shows the 'loadings' or 'weights' of each original feature on each principal component.

Since the PCA retained only one component, we'll look at how the original 39 features contribute to this single component. First, look at the the full list of feature names generated from the ColumnTransformer "ct", created as a result of One-Hot Encoding. And then look at the loadings to be able to interpret which original features are most influential in that principal component.

The output lists the 'loadings' of each of the original features onto the single principal component that PCA retained. These loadings indicate the correlation between the original features and the principal component.

From the sorted list, it's clear that "**TotalCharges**" has an extremely high absolute loading (nearly 1.0), which means the first principal component (PC1) is almost entirely explained by the TotalCharges feature. The other features have comparatively very small loadings, indicating they contribute much less to this principal component.

In essence, the data's primary variance, as captured by this single principal component, is overwhelmingly driven by the "**TotalCharges**" variable.



In [21]:
# Apply PCA for Dimension Reduction:
# The task requires applying PCA to the training and test datasets.
# The first step is to import the `PCA` class from `sklearn.decomposition`, instantiate it with the specified parameters,
# fit it to the training data, and then transform both the training and test datasets.
from sklearn.decomposition import PCA

# Instantiate PCA to retain 95% of the variance
pca = PCA(n_components=0.95, random_state=42)

# Fit PCA on the training data
pca.fit(X_train)

# Transform both training and test data
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

print(f"Original training data shape: {X_train.shape}")
print(f"Reduced training data shape: {X_train_pca.shape}")
print(f"Original test data shape: {X_test.shape}")
print(f"Reduced test data shape: {X_test_pca.shape}")
print(f"Number of components retained: {pca.n_components_}")

Original training data shape: (5625, 39)
Reduced training data shape: (5625, 1)
Original test data shape: (1407, 39)
Reduced test data shape: (1407, 1)
Number of components retained: 1


In [22]:
# Get full list of feature names from
# the ColumnTransformer "ct", created as a result of One-Hot Encoding.
def get_feature_names(column_transformer):
    output_features = []
    for name, estimator, features in column_transformer.transformers_:
        if name == 'passthrough':
            output_features.extend(features)
        elif hasattr(estimator, 'get_feature_names_out'):
            output_features.extend(estimator.get_feature_names_out(features))
        else:
            output_features.extend(features) # Fallback for estimators without get_feature_names_out
    return output_features

feature_names = get_feature_names(ct)

print("Original feature names after One-Hot Encoding and passthrough:")
print(feature_names)

Original feature names after One-Hot Encoding and passthrough:
['gender_Female', 'gender_Male', 'SeniorCitizen_0', 'SeniorCitizen_1', 'Partner_No', 'Partner_Yes', 'Dependents_No', 'Dependents_Yes', 'PhoneService_No', 'PhoneService_Yes', 'MultipleLines_No', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No', 'OnlineSecurity_Yes', 'OnlineBackup_No', 'OnlineBackup_Yes', 'DeviceProtection_No', 'DeviceProtection_Yes', 'TechSupport_No', 'TechSupport_Yes', 'StreamingTV_No', 'StreamingTV_Yes', 'StreamingMovies_No', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_No', 'PaperlessBilling_Yes', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check', 'tenure', 'MonthlyCharges', 'TotalCharges']


In [23]:
# Display the loadings of the single principal component
# The pca.components_ array has shape (n_components, n_features)
# Since n_components_ is 1, pca.components_[0] will give the loadings for
# the first (and only) principal component.

if pca.n_components_ > 0:
    loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(pca.n_components_)], index=feature_names)
    # Sort by absolute value to see the most influential features
    print("\nLoadings of original features on the first Principal Component (sorted by absolute value):")
    display(loadings['PC1'].abs().sort_values(ascending=False))
else:
    print("No principal components were retained.")


Loadings of original features on the first Principal Component (sorted by absolute value):


,PC1
TotalCharges,9.999224e-01
tenure,8.939627e-03
MonthlyCharges,8.664966e-03
StreamingMovies_Yes,1.120580e-04
StreamingMovies_No,1.120580e-04
StreamingTV_Yes,1.116870e-04
StreamingTV_No,1.116870e-04
DeviceProtection_Yes,1.092643e-04
DeviceProtection_No,1.092643e-04
OnlineBackup_Yes,1.060784e-04


We have successfully completed the data preprocessing, cleaning, one-hot encoding, Train/Test split and dimensionality reduction using PCA. The PCA results indicated that the "**TotalCharges**" feature contributes almost entirely to the single principal component retained.

Now that the data is prepared, the next logical step is to train machine learning models, such as Logistic Regression, Random Forest, or Gradient Boosting, and evaluate their performance on either the PCA-transformed dataset (i.e. X_train_pca, X_test_pca, y_train, y_test) or Untransformed dataset (i.e. X_train, X_test, y_train, y_test).